Official API Usage
==================

Since I've changed from using my homegrown API to my python interface to the official API, I'm going to redo the parsing and aggregating work I've done before, but with the different access. In addition, I am going to dump the pertinent JSON data into a saved repository so that I don't have to repull the data in the future, and create the CSV files I want to use as the basis of my analysis.

The JSON files will contain raw data, and the CSV files will be the processed data.

In [1]:
%matplotlib inline

import json

import matplotlib.pyplot as plt
import numpy
import pandas

import wcf

In [2]:
wcf_access = wcf.WCF(timeout=20)
wcf_access.load_and_connect('/home/mikemoran/bin/wcf/credentials.json')
assert wcf_access

In [3]:
games = wcf_access.get_draws_by_tournament(555)

In [4]:
len(games)

71

In [5]:
final = games[-1]

In [6]:
final

{'DrawInfo': {'DrawNumber': 23,
  'GameStart': '2016-04-10T15:00:00',
  'Id': 6109,
  'TournamentId': 555},
 'Ends': [{'EndNumber': 1,
   'GameId': 23974,
   'Id': 135604,
   'Team1': 0,
   'Team2': 1},
  {'EndNumber': 2, 'GameId': 23974, 'Id': 135605, 'Team1': 0, 'Team2': 0},
  {'EndNumber': 3, 'GameId': 23974, 'Id': 135606, 'Team1': 2, 'Team2': 0},
  {'EndNumber': 4, 'GameId': 23974, 'Id': 135607, 'Team1': 0, 'Team2': 0},
  {'EndNumber': 5, 'GameId': 23974, 'Id': 135608, 'Team1': 0, 'Team2': 2},
  {'EndNumber': 6, 'GameId': 23974, 'Id': 135609, 'Team1': 1, 'Team2': 0},
  {'EndNumber': 7, 'GameId': 23974, 'Id': 135610, 'Team1': 0, 'Team2': 0},
  {'EndNumber': 8, 'GameId': 23974, 'Id': 135611, 'Team1': 0, 'Team2': 0},
  {'EndNumber': 9, 'GameId': 23974, 'Id': 135612, 'Team1': 0, 'Team2': 2}],
 'GroupInformation': {'Id': 1, 'Name': 'Group A'},
 'Id': 23974,
 'PlayoffGameNumber': None,
 'Round': {'Abbreviation': 'F',
  'DrawNumber': 1000,
  'GameType': 4,
  'Id': 56,
  'Name': 'Final'},


Using this one game as a basic, let's build up a dictionary to "rehold" the information that I will actually want to keep. This process shouldn't be too hard, so that's nice.

In [7]:
def convert_game(game):
    team1 = {}
    team1['Percentage'] = game['Team1']['Percentage']
    team1['Total'] = game['Team1']['Result']
    team1['Name'] = game['Team1']['Team']['Name']
    team1['Code'] = game['Team1']['Team']['Code']
    team1['AssociationId'] = game['Team1']['Team']['AssociationId']
    team2 = {}
    team2['Percentage'] = game['Team2']['Percentage']
    team2['Total'] = game['Team2']['Result']
    team2['Name'] = game['Team2']['Team']['Name']
    team2['Code'] = game['Team2']['Team']['Code']
    team2['AssociationId'] = game['Team2']['Team']['AssociationId']

    ends = []
    for end in game['Ends']:
        this_end = {}
        this_end['EndNumber'] = end['EndNumber']
        this_end['Team1'] = end['Team1']
        this_end['Team2'] = end['Team2']
        ends.append(this_end)

    game_dict = {}
    game_dict['TournamentId'] = game['TournamentId']
    game_dict['LSFE'] = game['TossWinner']
    game_dict['Team1'] = team1
    game_dict['Team2'] = team2
    game_dict['Draw'] = game['Round']['Name']
    game_dict['Ends'] = ends
    return game_dict

# final_dict = convert_game(final)
# final_dict

In [8]:
game_dicts = [convert_game(g) for g in games]
# game_dicts

Now that we have a list of dictionaries holding all of the game data, let's save this in our data directory for later use.

In [9]:
with open('../data/555.json', 'w') as f:
    json.dump(game_dicts, f, indent=4)

OK, awesome! So, let's dump all of our JSON data now, then process it later.

In [ ]:
import time

failed = []
complete = []

for i in range(1, 556):
    try:
        games = wcf_access.get_draws_by_tournament(i)
        game_dicts = [convert_game(g) for g in games]
        with open('../data/{:03d}.json'.format(i), 'w') as f:
            json.dump(game_dicts, f, indent=2)
        complete.append(i)
    except:
        failed.append(i)
    time.sleep(5)